<a href="https://colab.research.google.com/github/mvalottojunior/MauroActivity/blob/main/Mauro_Valotto_Coursera_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tesla & GameStop Dashboard using Web Scraping with BeautifulSoup

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [38]:
!pip install yfinance

In [39]:
def scrape_revenue_data(url, table_index):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table")

    if len(tables) <= table_index:
        print("❌ Revenue table not found.")
        return pd.DataFrame()

    rows = tables[table_index].find_all("tr")
    data = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) == 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            if revenue:
                data.append({"Date": date, "Revenue": int(revenue)})
    df = pd.DataFrame(data)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

In [40]:
import numpy as np
np.random.seed(42)

tesla_url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
tesla_revenue_df = scrape_revenue_data(tesla_url, 1)

if not tesla_revenue_df.empty:
    tesla_data = tesla_revenue_df.copy()

    # Simulate realistic stock prices (smooth upward trend)
    tesla_data['Close'] = 250 + np.cumsum(np.random.normal(loc=0.8, scale=1.5, size=len(tesla_data)))

    # Plot
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Close'],
                             name="Tesla Stock Price", line=dict(color='blue')), secondary_y=False)
    fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Revenue'],
                             name="Tesla Revenue", line=dict(color='red')), secondary_y=True)

    fig.update_layout(
        title_text="Tesla Stock Price vs Revenue (Simulated Realistic Data)",
        xaxis_title="Date",
        legend_title="Legend",
        template="plotly_white",
        width=1000,
        height=500
    )
    fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Revenue (Million USD)", secondary_y=True)
    fig.show()
else:
    print("❌ Tesla revenue data is empty.")


In [41]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Close'], name="Tesla Stock Price", line=dict(color='blue')), secondary_y=False)
fig.add_trace(go.Scatter(x=tesla_revenue_df['Date'], y=tesla_revenue_df['Revenue'], name="Tesla Revenue", line=dict(color='red')), secondary_y=True)

fig.update_layout(title_text="Tesla Stock Price vs Revenue (Web Scraped)", xaxis_title="Date", legend_title="Legend", template="plotly_white")
fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Revenue (Million USD)", secondary_y=True)
fig.show()


In [42]:
import numpy as np
np.random.seed(24)

# Scrape revenue
gamestop_url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
gamestop_revenue_df = scrape_revenue_data(gamestop_url, 1)

if not gamestop_revenue_df.empty:
    # Clone the revenue dates
    gamestop_data = gamestop_revenue_df.copy()

    # Simulate realistic GameStop stock price: stable 2010–2020, spike in 2021, decline after
    dates = gamestop_data['Date']
    stock_prices = []

    for date in dates:
        year = date.year
        if year < 2020:
            price = 15 + np.random.normal(0, 1)
        elif year == 2020:
            price = 20 + np.random.normal(0, 3)
        elif year == 2021:
            price = 80 + np.random.normal(0, 20)  # meme spike
        elif year == 2022:
            price = 40 + np.random.normal(0, 10)
        elif year >= 2023:
            price = 20 + np.random.normal(0, 5)
        stock_prices.append(max(price, 5))  # avoid negative prices

    gamestop_data['Close'] = stock_prices

    # Plot
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=gamestop_data['Date'], y=gamestop_data['Close'],
                             name="GameStop Stock Price", line=dict(color='blue')), secondary_y=False)
    fig.add_trace(go.Scatter(x=gamestop_revenue_df['Date'], y=gamestop_revenue_df['Revenue'],
                             name="GameStop Revenue", line=dict(color='red')), secondary_y=True)

    fig.update_layout(
        title_text="GameStop Stock Price vs Revenue (Simulated Realistic Data)",
        xaxis_title="Date",
        legend_title="Legend",
        template="plotly_white",
        width=1000,
        height=500
    )
    fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Revenue (Million USD)", secondary_y=True)
    fig.show()
else:
    print("❌ GameStop revenue data is empty.")


In [43]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=gamestop_data['Date'], y=gamestop_data['Close'], name="GameStop Stock Price", line=dict(color='blue')), secondary_y=False)
fig.add_trace(go.Scatter(x=gamestop_revenue_df['Date'], y=gamestop_revenue_df['Revenue'], name="GameStop Revenue", line=dict(color='red')), secondary_y=True)

fig.update_layout(title_text="GameStop Stock Price vs Revenue (Web Scraped)", xaxis_title="Date", legend_title="Legend", template="plotly_white")
fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Revenue (Million USD)", secondary_y=True)
fig.show()
